### 调用百度API

In [5]:
from aip import AipNlp

APP_ID = '************'
API_KEY = '*******************'
SECRET_KEY = '***************************'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

### 读取数据

In [1]:
count = 0
alldata = []
with open("./all_docs.txt", "r", encoding="utf-8") as data:
    for line in data:
        line = line.strip()
        alldata.append(line)
        count += 1 
print(count)
print(len(alldata))

108295
108295


In [2]:
'''
    只读取训练集的title
'''
train_title_list = []
for line in alldata:
    line = line.strip()
    if len(line.split("\001")) == 3:
        title = line.split("\001")[1]
        train_title_list.append(title)
print(len(train_title_list))

108295


In [3]:
train_id_list = []
train_title_content_list = []
train_dict = {}  # key = id, value = title+"。"+content
for line in alldata:
    line = line.strip()
    if len(line.split("\001")) == 3:
        ids = line.split("\001")[0]
        title_content = line.split("\001")[1] + "。" + line.split("\001")[2]
        
        train_id_list.append(ids)
        train_title_content_list.append(title_content)
        
        train_dict[ids] = title_content

In [4]:
print(len(train_id_list))
print(len(train_title_content_list))
print(len(train_dict))

108295
108295
108295


### 提取title里《》中的内容，为下一版添加自定义分词做准备

In [68]:
'''
    自定义词典：
        ① 提取title中的《》中的内容    
'''

import re

results = []
tmp = []
with open("./user_dict_title.txt", "w", encoding="utf-8") as title_data:
    for count, title in enumerate(train_title_list):
        result = ' '.join(re.findall('\《(.*?)\》', title))
        if result:
            results.append(result)       # 16278

    for words in results:   # 4746
        for word in words.split(" "):
            if word not in tmp: # 记录已写进的《》,防止重复
                title_data.write(word + "\n")
                tmp.append(word)

### 转换自定义分词的编码为gbk，上传到百度

In [71]:
with open("./user_dict_title.txt", "r", encoding="utf-8") as data, open("./user_dict_title_gbk.txt", "w", encoding="GBK") as new_data:
    for line in data:
        line = line.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk').strip()
        new_data.write(line + "\n")   

#### 单独获取分词的结果，重复5次，加大现有结果中title的权重，已上传自定义词典

In [73]:
train_results = []
for num, line in enumerate(train_title_content_list[0:10000]):
    try:
        line = line.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk')
        result = client.lexerCustom(line)
        if result:
            cut_words = [item.get("item") for item in result.get("items")]
            train_results.append(" ".join(cut_words))
        else:train_results
            print(num)
            print("分词失败！")
    except Exception as e:
        print(e)
        print(num)

'NoneType' object is not iterable
4227


In [81]:
train_title_content_list[20]

'“后来的我们”4天票房破10亿，登上华语影史最最最最导演。小龙女拍视频认成龙是自己爸爸网易娱乐4月27日报道&nbsp;据香港媒体报道，吴绮莉女儿吴卓林与女友Andi与加拿大“寻爸”，录制视频短片显示为本月24日，该片更火速在网上疯传。原来Andi同日在网上平台上传了一段1分49秒的“求助”片段，视频中小龙女吴卓林和女友Andi面容憔悴，吴卓林在视频中表示，自己是JackieChan（成龙）的女儿，又叫大家转发片段，在走投无路下，小龙女终于公开认成龙为自己老爸。吴卓林在视频中表示拍片是想关心她的人知道发生什么事，她又说，“我们的爱比你们想像更强壮，但我们也感到疲累。没有人能帮到我们，包括我所认识的人、家人、朋友及政府机构”。她在网上平台又留言自爆患有创伤后压力症候群（PTSD）。吴卓林称自己曾报警，去过一个慈善机构，也去过医院和食物银行：“我们已经找过警察，去过医院、食物银行、LGBTQ庇护中心、但全部都不理我们。”吴卓林更指有人硬要将她与女友分开，更情绪激动：“两个相爱的人现在被人拆散，我不懂（为什么）。我爱她，我爱你（对女友说）。”说到此处更深情望向Andi表示爱她，Andi立马向吴卓林献吻。影片中，吴卓林与女友面容憔悴，看起来非常疲累，她在片中自称是成龙的女儿，两人已经在天桥下睡了近1个月，随后她的女友接着表示，自己已经找了所有的亲戚朋友，每个人都要她们去寻求庇护所的帮忙。但两人似乎非常抗拒到庇护所，称如果两人到了任何一个政府部门，或非政府组织都会被分开。'